# Create Neural Network for Image Inversion with numpy

### I. Read training data

In [10]:
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [13]:
# Convert PIL image to array normalised to 0 to 1 rage.
max_color = 255
def image_2_array(filename):
    pil_img=Image.open(filename)
    pil_arr=np.array(pil_img)
    pil_scaled_arr = (pil_arr / max_color)
    return pil_img, pil_scaled_arr

# Invert grayscale PIL image
def invert_array(arr):
    inverted_arr = 1 - arr
    return inverted_arr
    
# Convert normalized array to PIL image
def array_2_image(arr):
    arr_scaled = (arr) * max_color
    im = Image.fromarray(arr_scaled)
    return im

In [18]:
# Read training images into array of shape (m_samples, imsize, imsize, 1)
train_data_onedimension = np.array([])
train_labl_onedimension = np.array([])
DIR="imgset20x20x1000"
imsize=20
orig_pattern = DIR + "/origin*"
invr_pattern = DIR + "/inverse*"
import glob
for f in glob.glob(orig_pattern):
    img,a = image_2_array(f)
    train_data_onedimension = np.append(train_data_onedimension,a)
for f in glob.glob(invr_pattern):
    img,a = image_2_array(f)
    train_labl_onedimension = np.append(train_labl_onedimension,a)

print train_data_onedimension.shape
print train_labl_onedimension.shape

(400000,)
(400000,)


In [19]:
m_samples = train_data_onedimension.shape[0] / imsize / imsize
train_data = train_data_onedimension.reshape(m_samples, imsize, imsize, 1)
train_labl = train_labl_onedimension.reshape(m_samples, imsize, imsize, 1)
print train_data.shape
print train_labl.shape


(1000, 20, 20, 1)
(1000, 20, 20, 1)


In [21]:
# Split data between training and test sets 700 : 300
TEST_SIZE = 300
TRAINING_SIZE = m_samples - TEST_SIZE

test_data  = train_data[:TEST_SIZE,:,:,:]
train_data = train_data[TEST_SIZE:,:,:,:]

test_labl  = train_labl[:TEST_SIZE,:,:,:]
train_labl = train_labl[TEST_SIZE:,:,:,:]
print("Dataset dimentions:")
print train_data.shape
print train_labl.shape
print test_data.shape
print test_labl.shape
diff_matr = train_data[1]-train_labl[1]
print diff_matr[:2,:2,:]

Dataset dimentions:
(700, 20, 20, 1)
(700, 20, 20, 1)
(300, 20, 20, 1)
(300, 20, 20, 1)
[[[ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]]]
